# ✌가위바위보 분류기

In [1]:
from PIL import Image
import os, glob
import numpy as np

import cv2

import matplotlib.pyplot as plt

# 이미지를 일정한 크기로 변환하는 함수
- 불필요한 변환을 막기 위해 

In [2]:
def resize_images(img_path):
    images = glob.glob(img_path + "/*.jpg") # 해당 형식의 이름을 가진 파일의 리스트
    
    target_size = (28, 28)
    for img in images:
        old_img = Image.open(img)
        #불필요한 연산 미시행
        if old_img.size == target_size:
            continue
        new_img = old_img.resize(target_size, Image.ANTIALIAS) #ANTIALIAS ???
        new_img.save(img,'JPEG')
    print(len(images), "Image resized to", target_size)

# 데이터 경로 지정 및 크기조정

In [3]:
train_path = ['./data/rock_scissor_paper/train/paper',
       './data/rock_scissor_paper/train/scissor',
       './data/rock_scissor_paper/train/rock']
test_path = ['./data/rock_scissor_paper/test/paper',
       './data/rock_scissor_paper/test/scissor',
       './data/rock_scissor_paper/test/rock']
for fp in train_path:
    resize_images(fp)
for fp in test_path:
    resize_images(fp)

1108 Image resized to (28, 28)
994 Image resized to (28, 28)
1105 Image resized to (28, 28)
100 Image resized to (28, 28)
100 Image resized to (28, 28)
100 Image resized to (28, 28)


# 이미지 불러오는 함수 정의
- 가위, 바위, 보 각 파일의 크기가 달라도 동적으로 불러올 수 있도록 수정
- 성능 향상을 위한 graysclae 적용

In [4]:
def load_data(img_path):
    img_size = 28
    #garysacale을 활용하기 위해 color=1, dtype=np.uint8로 처리함
    color = 1
    
    # 개수가 다른 가위, 바위, 보가 입력되더라도 처리할 수 있도록 변경
    imgs = np.zeros(img_size * img_size * color, dtype = np.uint8).reshape(-1,img_size,img_size,color)
    labels = np.zeros(1, dtype=np.uint8)
        
    for idx, file in enumerate(glob.iglob(img_path+'/scissor/*jpg')) :
        img = np.array(Image.open(file).convert('L'), dtype = np.uint8).reshape(-1, img_size, img_size, color)
        imgs = np.append(imgs, img, axis = 0)
        labels = np.append(labels, 0)
        
    for idx, file in enumerate(glob.iglob(img_path+'/rock/*jpg')) :
        img = np.array(Image.open(file).convert('L'), dtype = np.uint8).reshape(-1, img_size, img_size, color)
        imgs = np.append(imgs, img, axis = 0)
        labels = np.append(labels, 1)

    for idx, file in enumerate(glob.iglob(img_path+'/paper/*jpg')) :
        img = np.array(Image.open(file).convert('L'), dtype = np.uint8).reshape(-1, img_size, img_size, color)
        imgs = np.append(imgs, img, axis = 0)
        labels = np.append(labels, 2)
    
    #틀을 잡아주었던 첫 행을 제외하고 실제 데이터가 있는 부분만 취함
    imgs = imgs[1:]
    labels = labels[1:]
    
    return imgs, labels

# 데이터를 불러오기
- aiffel에 공유된 3000여 개의 자료로 train하고 직접 만든 가위바위도 300개로 test하도록 데이터를 준비하였음

In [5]:
image_dir_path_train = './data/rock_scissor_paper/train'
image_dir_path_test = './data/rock_scissor_paper/test'

x_train, y_train = load_data(image_dir_path_train)
x_test, y_test = load_data(image_dir_path_test)

x_train_norm = x_train / 255.0
x_test_norm = x_test / 255.0

#이미 원하는 차원을 가졌으므로 reshape 불필요
# x_train_reshaped = x_train_norm.reshape(-1, 28, 28, 3)
# x_test_reshaped = x_test_norm.reshape(-1, 28, 28, 3)

print(f"x_train_shape: {x_train_norm.shape}")
print(f"y_train_shape: {y_train.shape}")
print(f"x_test_shape: {x_test_norm.shape}")
print(f"y_test_shape: {y_test.shape}")

x_train_shape: (3207, 28, 28, 1)
y_train_shape: (3207,)
x_test_shape: (300, 28, 28, 1)
y_test_shape: (300,)


# 학습층 정의
- 은닉층의 수를 늘리고, filter수를 64개에서 32개로 줄여 파라미터 수는 줄이되 성능은 높임

In [6]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.Sequential( )
model.add(keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape=(28,28,1) ) )
model.add(keras.layers.MaxPool2D((2,2)) )
model.add(keras.layers.Conv2D(32, (3,3), activation = 'relu') )
model.add(keras.layers.MaxPool2D((2,2)) )
model.add(keras.layers.Conv2D(32, (3,3), activation = 'relu') )
model.add(keras.layers.MaxPool2D((2,2)) )

model.add(keras.layers.Flatten() )

model.add(keras.layers.Dense(32, activation = 'relu') )
model.add(keras.layers.Dense(32, activation = 'relu') )
model.add(keras.layers.Dense(3, activation='softmax') )

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 32)          9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 32)                0

In [7]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(x_train_norm, y_train, epochs = 20 )

Epoch 1/20
101/101 [==============================] - 3s 21ms/step - loss: 1.0861 - accuracy: 0.3941
Epoch 2/20
101/101 [==============================] - 2s 22ms/step - loss: 0.9718 - accuracy: 0.5076
Epoch 3/20
101/101 [==============================] - 2s 21ms/step - loss: 0.7554 - accuracy: 0.6651
Epoch 4/20
101/101 [==============================] - 2s 22ms/step - loss: 0.5804 - accuracy: 0.7527
Epoch 5/20
101/101 [==============================] - 2s 20ms/step - loss: 0.4498 - accuracy: 0.8238
Epoch 6/20
101/101 [==============================] - 2s 25ms/step - loss: 0.3752 - accuracy: 0.8544
Epoch 7/20
101/101 [==============================] - 3s 30ms/step - loss: 0.3202 - accuracy: 0.8778
Epoch 8/20
101/101 [==============================] - 3s 30ms/step - loss: 0.2556 - accuracy: 0.9021
Epoch 9/20
101/101 [==============================] - 3s 32ms/step - loss: 0.2286 - accuracy: 0.9171 
Epoch 10/20
101/101 [==============================] - 3s 33ms/step - loss: 0.1824 - accur

# 결론
- 완전히 분리된 train과 test셋으로 0.71의 정확도를 보임

test_loss, test_accuracy = model.evaluate(x_test_norm, y_test)
print(test_loss, test_accuracy)

# 회고
- 여러 조건에 따라 accuracy의 variance가 상당히 높았으나 그 이유에 대한 명확한 지식이 없어 어려움을 겪었음
- 정적으로 정의된 함수를 동적으로 수행하도록 바꾸면서 코드를 깊이 있게 이해할 수 있었고, 익숙하지 않은 tensorlow를 여러 방향으로 수정해 보면서 직관적인 감각을 느낄 수 있었음
- 루브릭 평가 지표를 달성하기 위해 충분한 데이터를 확보하고, GrayScale 변환과 은닉층을 여러차례 수정하여 평가하였음
- 그러나 근본적인 이해가 없는 상태에서 지나치게 세밀하게 파고드는 것은 자칫 비효율성을 초래할 수 있음을 느낌
- 큰 시야에서 다양한 시도를 할 수 있는 균형을 찾는 것이 중요하다고 생각함